In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from scipy.optimize import curve_fit
from scipy.optimize import linear_sum_assignment
import tqdm



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
LS133 = "./ntupleStubsBmtfGmt_LS133.root"
gmt_awk_LS133 = OpenRootFile(LS133, gmt_features)
bmtf_awk_LS133 = OpenRootFile(LS133, bmtf_features)

In [33]:
Gmt_Phi_LS133 = gmt_awk_LS133.gmtPhi
Bmtf_Phi_LS133 = bmtf_awk_LS133.bmtfPhi

Gmt_Eta_LS133 = gmt_awk_LS133.gmtEta
Bmtf_Eta_LS133 = bmtf_awk_LS133.bmtfEta

nGmtMuons_LS133 = gmt_awk_LS133.nGmtMuons
nBmtfMuons_LS133 = bmtf_awk_LS133.nBmtfMuons



# Definire l'array
array = [[1, 2, 3], 2, 3, 4]
array = ak.Array(array)

i = 0
j = 1

# Crea una nuova versione di array[i] senza l'elemento j
array_new = ak.concatenate([array[i][:j], array[i][j+1:]])

# Ricostruisci l'intero array con la modifica
new_array = ak.Array([array_new] + array[1:])

print(array_new)
print(new_array)
right = ak.concatenate(new_array, array[1:])

[1, 3]
[[3, 5], [4, 6], [5, 7]]


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Array'

In [4]:
def Delta(BMTF, GMT):
  return BMTF - GMT

def DeltaR(BmtfPhi, GmtPhi, BmtfEta, GmtEta):
  return np.sqrt((BmtfPhi - GmtPhi)**2 + (BmtfEta - GmtEta)**2)

In [21]:
MuonsProperties = []

use_Bmtf = True


for i in tqdm.tqdm(range(50000)):

  DeltaRDistBMTF = []
  IndexMinBMTF = []
  DeltaRDistGMT = []
  IndexMinGMT = []

  if nGmtMuons_LS133[i] == 0 and nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = []
  
  elif nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = [range(nGmtMuons_LS133[i])]

  elif nGmtMuons_LS133[i] == 0:
    unmatched_Bmtf = [range(nBmtfMuons_LS133[i])]
    unmatched_Gmt = []

  else:
    #Primo metodo 
    for j in range(nBmtfMuons_LS133[i]):

      DeltaRTemp = DeltaR(Bmtf_Phi_LS133[i, j], Gmt_Phi_LS133[i, :], Bmtf_Eta_LS133[i, j], Gmt_Eta_LS133[i, :])

      minDeltaR = np.min(DeltaRTemp)

      if minDeltaR < 0.4:

        indexTempBMTF = np.argmin(DeltaRTemp)
        DeltaRDistBMTF.append(minDeltaR)
        IndexMinBMTF.append(indexTempBMTF)
        DeltaPhi_BMTF = Delta(Bmtf_Phi_LS133[i, j], Gmt_Phi_LS133[i, indexTempBMTF])
        DeltaEta_BMTF = Delta(Bmtf_Eta_LS133[i, j], Gmt_Eta_LS133[i, indexTempBMTF])

      else:
        #Se DeltaR > 0.4 (primo metodo) non c'è match con i muoni del GMT
        unmatched_Gmt = [range(nGmtMuons_LS133[i])]
  
    
      matched_Bmtf_Index = [i for i in IndexMinBMTF if IndexMinBMTF]
      unmatched_Bmtf = [i for i in range(nBmtfMuons_LS133[i]) if i not in matched_Bmtf_Index]


    
    #Secondo metodo 
    for k in range(nGmtMuons_LS133[i]):
      DeltaRTemp = DeltaR(Bmtf_Phi_LS133[i, :], Gmt_Phi_LS133[i, k], Bmtf_Eta_LS133[i, :], Gmt_Eta_LS133[i, k])

      minDeltaR = np.min(DeltaRTemp)

      if minDeltaR < 0.4:
  
        indexTempGMT = np.argmin(DeltaRTemp)
        DeltaRDistGMT.append(minDeltaR)
        IndexMinGMT.append(indexTempGMT)
        DeltaPhi_GMT = Delta(Bmtf_Phi_LS133[i, indexTempGMT], Gmt_Phi_LS133[i, k])
        DeltaEta_GMT = Delta(Bmtf_Eta_LS133[i, indexTempGMT], Gmt_Eta_LS133[i, k])
      
      else:
        unmatched_Bmtf = [range(nBmtfMuons_LS133[i])]

      matched_Gmt_Index = [k for k in IndexMinGMT if IndexMinGMT]
      unmatched_Gmt = [k for k in range(nGmtMuons_LS133[i]) if k not in matched_Gmt_Index]


      #Tolgo il valore 

  Events = {
    'unmatched_Gmt_Events' : unmatched_Gmt,
    'unmatched_Bmtf_Events': unmatched_Bmtf, 
    'DeltaR': DeltaRDistBMTF,
  }

  MuonsProperties.append(Events)


100%|██████████| 50000/50000 [00:30<00:00, 1647.53it/s]


In [22]:
MuonsProperties = ak.Array(MuonsProperties)
print(ak.sum(ak.num(MuonsProperties.unmatched_Bmtf_Events)))
print(ak.sum(ak.num(MuonsProperties.unmatched_Gmt_Events)))

554
94
